In [57]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf 

import matplotlib.image as mpimg

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

In [54]:
TRAIN_DIR = 'datasets/cat_dog/train'
TEST_DIR = 'datasets/cat_dog/test'

print(len(os.listdir(TRAIN_DIR)),len(os.listdir(TEST_DIR)))

train_image_file_names = [TRAIN_DIR + '/' + i for i in os.listdir(TRAIN_DIR)][12000:13000]

25000 12500


In [55]:
def create_data(image_file_names, width=500, height=500, final_width=214, final_height=214):
    import scipy.misc
    
    def resize_width_height(image, width=width, height=width, final_width=final_width, final_height=final_height):
        tmp = np.zeros((width,height,3))
        tmp[:image.shape[0], :image.shape[1], :] = image
        tmp = scipy.misc.imresize(tmp, (final_width,final_height))
        tmp = tmp/ 255
        return tmp.reshape(-1, final_width, final_height, 3) ## batch, width, height, channel
    
    data = np.zeros((1, final_width, final_height, 3))
    num = 0
    for i in image_file_names:
        tmp = mpimg.imread(i)
        data = np.r_[data, resize_width_height(tmp)]
        if num % 10 == 0:
            print("아직 문제 없습니다.", num, end='\r')
        num += 1
    return data[1:,:,:,:]

In [59]:
train_data0 = create_data(train_image_file_names[:500]) 
## train_image_file은 이미지 주소가 있는 str입니다.
# test_data0 = create_data(test_image_file_names[:500])

train_data1 = create_data(train_image_file_names[500:]) 
## train_image_file은 이미지 주소가 있는 str입니다.
#test_data1 = create_data(test_image_file_names[500:])

train_data = np.r_[train_data0, train_data1]
# test_data = np.r_[test_data0, test_data1]

c:\python\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  import sys


In [61]:
train_data.shape

(1000, 214, 214, 3)

In [62]:
labels = np.array([[1., 0.] if 'dog' in name[23:] else [0., 1.] for name in train_image_file_names])
labels.shape

(1000, 2)

In [63]:
reset_graph()

In [64]:
X_image = tf.placeholder("float", shape=[None, 214, 214, 3])
y = tf.placeholder("float", shape=[None, 2])

가중치를 표준편차 0.1을 갖는 난수로 초기화하는 함수와 바이어스를 0.1로 초기화하는 함수를 정의합니다.

In [65]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

stride는 1로 하고 패딩은 0으로 하는 합성곱층을 만드는 함수와 2 $\times$ 2 최대 풀링 레이어와 평균 풀링 레이어를 위한 함수를 정의합니다.

In [66]:
def conv2d(x, W, strides=[1,1,1,1], padding="SAME"):
    return tf.nn.conv2d(x, W, strides=strides, padding=padding)

def max_pool_3x3(x, padding="SAME"):
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,2,2,1], padding=padding)

def avg_pool_7x7(x, padding="VALID"):
    return tf.nn.avg_pool(x, ksize=[1,7,7,1], strides=[1,1,1,1], padding=padding)

첫 번째 합성곱층을 만들기 위해 가중치와 바이어스 텐서를 만들고, 활성화함수는 ReLU 함수를 사용합니다. 그리고 합성곱층 뒤에 최대 풀링층을 추가합니다.

In [67]:
W_conv1 = weight_variable([7,7,3,64]) ## 수용장 너비, 수용장 높이, 컬러, 특성 맵 개수
b_conv1 = bias_variable([64])

h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1, strides=[1,2,2,1]) + b_conv1)

h_pool1 = max_pool_3x3(h_conv1)

lrn_conv1 = tf.nn.lrn(h_pool1, depth_radius=2, bias=1,
                          alpha=0.00002, beta=0.75)

두 번째 합성곱층과 세 번쨰 합성곱층을 만들기 위해 가중치와 바이어스 텐서를 만들고, 활성화함수는 ReLU 함수를 사용합니다. 그리고 합성곱층 뒤에 정규화 함수를 추가한 후 최대 풀링층을 추가합니다.

In [68]:
W_conv2 = weight_variable([1,1,64,64]) ## 수용장 너비, 수용장 높이, 컬러, 특성 맵 개수
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(lrn_conv1, W_conv2, strides=[1,1,1,1]) + b_conv2)

W_conv3 = weight_variable([3,3,64,192]) ## 수용장 너비, 수용장 높이, 컬러, 특성 맵 개수
b_conv3 = bias_variable([192])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, strides=[1,1,1,1]) + b_conv3)

lrn_conv2 = tf.nn.lrn(h_conv3, depth_radius=2, bias=1,
                          alpha=0.00002, beta=0.75)

h_pool2 = max_pool_3x3(lrn_conv2)

이제 인셉션 모듈을 만들어보겠습니다. 인셉션 모듈을 편하게 사용하기위해 함수로 만들겠습니다.

In [69]:
def inception(x, in_channels, filter_count):
    # 1x1 1
    W_incep1  = weight_variable([1,1,in_channels, filter_count[0]])
    b_incep1 = bias_variable([filter_count[0]])
    one_by_one1 = tf.nn.relu(conv2d(x, W_incep1, strides=[1,1,1,1]) + b_incep1)
    
    # 1x1 2, 3x3 
    W_incep2  = weight_variable([1,1,in_channels, filter_count[1]])
    b_incep2 = bias_variable([filter_count[1]])
    one_by_one2 = tf.nn.relu(conv2d(x, W_incep2, strides=[1,1,1,1]) + b_incep2)
    
    W_incep3x3  = weight_variable([3,3,filter_count[1], filter_count[2]])
    b_incep3x3 = bias_variable([filter_count[2]])
    three_by_three = tf.nn.relu(conv2d(one_by_one2, W_incep3x3, strides=[1,1,1,1]) + b_incep3x3)
    
    # 1x1 3, 5x5
    W_incep3  = weight_variable([1,1,in_channels, filter_count[3]])
    b_incep3 = bias_variable([filter_count[3]])
    one_by_one3 = tf.nn.relu(conv2d(x, W_incep3, strides=[1,1,1,1]) + b_incep3)
   
    W_incep5x5  = weight_variable([5,5,filter_count[3], filter_count[4]])
    b_incep5x5 = bias_variable([filter_count[4]])
    five_by_five = tf.nn.relu(conv2d(one_by_one3, W_incep5x5, strides=[1,1,1,1]) + b_incep5x5)
   
    # max pool, 1x1 4
    max_pool = tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME')

    W_incep4  = weight_variable([1,1,in_channels, filter_count[5]])
    b_incep4 = bias_variable([filter_count[5]])
    one_by_one4 = tf.nn.relu(conv2d(max_pool, W_incep4, strides=[1,1,1,1]) + b_incep4)
    
    ## concat
    
    x = tf.concat([one_by_one1, three_by_three, five_by_five, one_by_one4], axis=3)
    
    return tf.nn.relu(x)

1,2 번째 인셉션 모듈을 만들겠습니다. 그리고 최대 풀링층을 추가합니다.

In [70]:
h_incep1 = inception(h_pool2, in_channels=192, filter_count=[64,96,128,16,32,32])
h_incep2 = inception(h_incep1, in_channels=64+128+32+32, filter_count=[128,128,192,32,96,64])

h_pool3 = max_pool_3x3(h_incep2)
h_pool3.get_shape()

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(480)])

3,4,5,6,7 번째 인셉션 모듈을 추가하겠습니다. 그리고 최대 풀링층을 추가합니다.

In [71]:
h_incep3 = inception(h_pool3, in_channels=480, filter_count=[192,96,208,16,48,64])
h_incep4 = inception(h_incep3, in_channels=192+208+48+64, filter_count=[160,112,224,24,64,64])
h_incep5 = inception(h_incep4, in_channels=160+224+64+64, filter_count=[128,128,256,24,64,64])
h_incep6 = inception(h_incep5, in_channels=128+256+64+64, filter_count=[112,144,288,32,64,64])
h_incep7 = inception(h_incep6, in_channels=112+288+64+64, filter_count=[256,160,320,32,128,128])

h_pool4 = max_pool_3x3(h_incep7)
h_pool4.get_shape()

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(832)])

8, 9 번째 인셉션 모듈을 추가하겠습니다. 그리고 평균 풀링층을 추가합니다.

In [72]:
h_incep8 = inception(h_pool4, in_channels=832, filter_count=[256,160,320,32,128,128])
h_incep9 = inception(h_incep8, in_channels=256+320+128+128, filter_count=[384,192,384,48,128,128])

h_pool5 = avg_pool_7x7(h_incep9)
h_pool5.get_shape()

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(1024)])

드랍아웃 층을 추가하겠습니다.

In [73]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_pool5, keep_prob)

완전 연결층을 추가합니다.

In [74]:
W_fc1 = weight_variable([1 * 1 * 1024, 1000]) ## 4096는 뉴런 개수
b_fc1 = bias_variable([1000])

h_fc1_drop_flat = tf.reshape(h_fc1_drop, [-1, 1 * 1 * 1024])
h_fc1 = tf.nn.relu(tf.matmul(h_fc1_drop_flat, W_fc1) + b_fc1)

softmax층과 연결합니다.

In [75]:
W_fc2 = weight_variable([1000, 2])
b_fc2 = bias_variable([2])

k = tf.matmul(h_fc1, W_fc2) + b_fc2
y_conv = tf.nn.softmax(k)

손실함수는 cross entropy로 하겠습니다.

In [78]:
xentropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=y, logits=k)
loss = tf.reduce_mean(xentropy, name="loss")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



러닝 알고리즘을 정의하겠습니다.

In [79]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_step = optimizer.minimize(loss)

이제 정확도 함수를 정의하겠습니다.

In [80]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

이제 신경망을 훈련 시켜보겠습니다.

In [81]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [82]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 30
batch_size = 25

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        num = 0
        for X_batch, y_batch in shuffle_batch(train_data, labels, batch_size):
            _, y_pred = sess.run([train_step, y_conv], feed_dict={X_image: X_batch, y: y_batch, keep_prob: 0.5})
            if num % 10 == 0:
                acc_batch = accuracy.eval(feed_dict={X_image: X_batch, y: y_batch, keep_prob: 0.5})
                print(num, "미니 배치 데이터 정확도:", acc_batch)
            num += 1
        
        acc_batch = accuracy.eval(feed_dict={X_image: X_batch, y: y_batch, keep_prob: 0.4})
        print('epoch', epoch, "인 데이터 정확도:", acc_batch)        
        
    save_path = saver.save(sess, "./cnn_alexnet.ckpt")

0 미니 배치 데이터 정확도: 0.68
5 미니 배치 데이터 정확도: 0.68
10 미니 배치 데이터 정확도: 0.6
15 미니 배치 데이터 정확도: 0.68
20 미니 배치 데이터 정확도: 0.48
25 미니 배치 데이터 정확도: 0.56
30 미니 배치 데이터 정확도: 0.36
35 미니 배치 데이터 정확도: 0.44
epoch 0 인 데이터 정확도: 0.44
0 미니 배치 데이터 정확도: 0.52
5 미니 배치 데이터 정확도: 0.44
10 미니 배치 데이터 정확도: 0.36
15 미니 배치 데이터 정확도: 0.32
20 미니 배치 데이터 정확도: 0.44
25 미니 배치 데이터 정확도: 0.4
30 미니 배치 데이터 정확도: 0.4
35 미니 배치 데이터 정확도: 0.68
epoch 1 인 데이터 정확도: 0.6
0 미니 배치 데이터 정확도: 0.36
5 미니 배치 데이터 정확도: 0.6
10 미니 배치 데이터 정확도: 0.32
15 미니 배치 데이터 정확도: 0.4
20 미니 배치 데이터 정확도: 0.52
25 미니 배치 데이터 정확도: 0.56
30 미니 배치 데이터 정확도: 0.6
35 미니 배치 데이터 정확도: 0.56
epoch 2 인 데이터 정확도: 0.72
0 미니 배치 데이터 정확도: 0.56
5 미니 배치 데이터 정확도: 0.4
10 미니 배치 데이터 정확도: 0.44
15 미니 배치 데이터 정확도: 0.52
20 미니 배치 데이터 정확도: 0.6
25 미니 배치 데이터 정확도: 0.48
30 미니 배치 데이터 정확도: 0.36
35 미니 배치 데이터 정확도: 0.32
epoch 3 인 데이터 정확도: 0.48
0 미니 배치 데이터 정확도: 0.36
5 미니 배치 데이터 정확도: 0.44
10 미니 배치 데이터 정확도: 0.6
15 미니 배치 데이터 정확도: 0.56
20 미니 배치 데이터 정확도: 0.56
25 미니 배치 데이터 정확도: 0.4
30 미니 배치 데이터 정확도: 0.52
35 미니 배치 데이터 정확도: 0.76
epoch

KeyboardInterrupt: 